In [ ]:
import deepxde as dde
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
from deepxde.backend import tf
import shutil
import os
from scipy import interpolate
import pandas as pd

In [ ]:
alpha = 9.81
kappa = 1
v0 = 1
a = 0.2
b = 0.3
L =  0.25

# Formulation of the Problem
Modified system of equations can be written as follows:

\begin{gather*}
\frac{dv}{dt} = A \\
\frac{dA}{dt} = \kappa(v_o - v) - \alpha\left(\frac{aA}{v} + \frac{b}{\theta}\left(1 - \frac{v\theta}{L}\right)\right) \\
\frac{d\theta}{dt} = 1 - \frac{v\theta}{L}
\end{gather*}


In [ ]:
def ode_system(x, y):
  v = y[:, 0:1]
  A = y[:, 1:2]
  theta = y[:, 2:3]

  dv_t = dde.grad.jacobian(y, x, i=0)
  dA_t = dde.grad.jacobian(y, x, i=1)
  dtheta_t = dde.grad.jacobian(y, x, i=2)

  return   [
      dv_t - A,
      dA_t - kappa * (v0 - v) + alpha * (a/v*A + b/theta*(1-v*theta/L)),
      dtheta_t - 1 + v*theta/L ]

In [ ]:
def MSE(pred1, pred2):
  return np.square(np.subtract(pred1, pred2)).mean()

# Define Training Function
`train_deepxde` is the training function for each time step. It takes number of residual points, time, initial values of $v$, $A$, and $\theta$. The default traininf has 4 layer of 64 neurons, Adam optimizer with learning rate 0.001. We train for 30,000 iterations.

In [ ]:
def train_deepxde(num_res, tmax, v0, A0, theta0, model_index):
  def output_transform(t, y):
    y1 = y[:, 0:1]
    y2 = y[:, 1:2]
    y3 = y[:, 2:3]

    return tf.concat(
        [y1 * tf.tanh(t) + v0, y2 * tf.tanh(t) + A0, y3 * tf.tanh(t) + theta0], axis=1
    )
  geom = dde.geometry.TimeDomain(0, tmax)
  data = dde.data.PDE(geom, ode_system,[], num_res, 0, num_test = 30000)
  layer_size = [1] + [64] * 4 + [3]
  activation = "tanh"
  initializer = "Glorot normal"
  net = dde.nn.FNN(layer_size, activation, initializer)
  net.apply_output_transform(output_transform)
  model = dde.Model(data, net)
  model.compile("adam", lr=0.001)
  checker = dde.callbacks.ModelCheckpoint(
      "Model/"+str(model_index)+"/model"+str(count)+"/model.ckpt", save_better_only=True, period=500
  )
  losshistory, train_state = model.train(iterations=30000, callbacks=[checker])


  if losshistory.steps[-1] != 30000:
      model, train_state = train_deepxde(num_res, tmax, v0, A0, theta0)
  else:
      dde.saveplot(losshistory, train_state, issave=True, isplot=True)
      model.restore("Model/"+str(model_index)+"/model"+str(count)+"/model.ckpt-" + str(train_state.best_step)+'.ckpt', verbose=1)
      dde.saveplot(losshistory, train_state, issave=True, isplot=True)


  return model

# Train Adaptive Time Stepping with MSE Tolerance
We train 3 models and inspect MSE of them. The training is 1 full step and 2 half steps. If MSE is greater than the tolerance, we bisect time step and train the model again. If not, the model proceed to the next step.

In [ ]:
tol = 0.0001
tmax = 10
num_res = 100000

t_values = []
v_values = []
A_values = []
theta_values = []
t_log = []
v_log = []
A_log = []
theta_log = []

t = 9.75
v, A, theta = 0.026057033,0.008191966,4.470924
count = 0

while t <= tmax:
  t_values.append(t)
  v_values.append(v)
  A_values.append(A)
  theta_values.append(theta)
  h = 0.5

  trunc_error = 100
  while trunc_error > tol:
    dt = 2*h

    model1_half = train_deepxde(num_res, h, v, A, theta, 1)
    t_test1 = np.linspace(0, h, 1000).reshape(1000, 1)
    pred1_half = model1_half.predict(t_test1)
    v1_half = pred1_half[:,0].reshape(-1)
    A1_half = pred1_half[:,1].reshape(-1)
    theta1_half = pred1_half[:,2].reshape(-1)

    model1_end = train_deepxde(num_res, h, v1_half[-1], A1_half[-1], theta1_half[-1], 2)
    pred1 = model1_end.predict(t_test1)
    v1_end = pred1[:,0].reshape(-1)
    A1_end = pred1[:,1].reshape(-1)
    theta1_end = pred1[:,2].reshape(-1)

    v1 = np.concatenate((v1_half, v1_end), axis = 0)
    A1 = np.concatenate((A1_half, A1_end), axis = 0)
    theta1 = np.concatenate((theta1_half, theta1_end), axis = 0)

    model2 = train_deepxde(num_res, 2*h, v, A, theta, 3)
    t_test2 = np.linspace(0, 2*h, 2000).reshape(2000, 1)
    pred2 = model2.predict(t_test2)
    v2 = pred2[:,0].reshape(-1)
    A2 = pred2[:,1].reshape(-1)
    theta2 = pred2[:,2].reshape(-1)

    mse_v = MSE(v1, v2)
    mse_A = MSE(A1, A2)
    mse_theta = MSE(theta1, theta2)

    trunc_error = np.sqrt(mse_v**2 + mse_A**2 + mse_theta**2)

    h = h/2

  v = v1[-1]
  A = A1[-1]
  theta = theta1[-1]
  t1 = np.linspace(t, t+dt, 2000)
  t += dt
  count += 1

  def output_transform(t, y):
      y1 = y[:, 0:1]
      y2 = y[:, 1:2]
      y3 = y[:, 2:3]

      return tf.concat(
          [y1 * tf.tanh(t) + v1[-1], y2 * tf.tanh(t) + A1[-1], y3 * tf.tanh(t) + theta1[-1]], axis=1
      )


  t_log.append(t1)
  v_log.append(v1)
  A_log.append(A1)
  theta_log.append(theta1)

  dic1 = {'t_train': np.array(t_log).reshape(-1), 'pred_v': np.array(v_log).reshape(-1), 'pred_A': np.array(A_log).reshape(-1), 'pred_t': np.array(theta_log).reshape(-1)}
  dic2 = {'t_step': np.array(t_values).reshape(-1), 'v_step': np.array(v_values).reshape(-1), 'A_step': np.array(A_values).reshape(-1), 'theta_step': np.array(theta_values).reshape(-1)}

  df1 = pd.DataFrame(dic1)
  df2 = pd.DataFrame(dic2)

  df1.to_csv('Prediction/pred_'+str(t)+'.csv')
  df2.to_csv('Prediction/pred_step_'+str(t)+'.csv')